# Kalman filters For Self Driving Vehicles

Explained the intuition and reasoning behind the user of Kalman Filters for self driving vehicles. Also Implemented Kalman Filters to predict object's movements and velocity

## Reasoning for Kalman Filters

We know that localization can be used with sensors to find out where our car is in the world.

Kalman filters can be used to predict incoming cars and traffic while the self driving car is driving. A self driving car uses lasers and radars to track other vehicles.

Kalman filters are useful because it can be used to find other cars and predict their velocity and movement which is important because with this information we can drive in a way where we do not crash into them. It can also be used for pedestrians and bicyclists

<img src="http://drive.google.com/uc?export=view&id=1b3HVRMHSScx3H7Lm09VzzvVSeWP2FiwQ">

**Figure of Kalman Filters predicting where the object will appear next. In the context of self driving cars, this can be vehicles, pedestrians, bicycles etc.**

## Kalman Filters vs Monte Carlo Localization

    - Estimates a continuous state as opposed to a discrete state in Monte Carlo Localization (choppping the world up into discrete places.)
    
    - Uni-Modal distribution as opposed to a Multi-modal distribution


<img src="http://drive.google.com/uc?export=view&id=112hWTnHgY8BV3kvM9nlhxUM4_CJ8g1G1">

## One dimensional Kalman Filter implementation

### Update function

Function responsible for updating the measurements for the gaussian distribution (multiplication of Gaussians)

In [100]:
def update(mean1, var1, mean2, var2):
    new_mean = (var1 * mean2 + var2 * mean1) / (var1 + var2)
    new_var = (1 / ((1/var1) + (1/var2)))
    return new_mean, new_var

In [101]:
print(update(10., 4., 12., 4.))

(11.0, 2.0)


This is the correct behavior, since the mean of the two Gaussians are 10 and 12 respectively with the same variance. This means that the new resulting Gaussian should be in the middle of these two, and with a decreased variance

### Predict function

Function responsible to predict the movement of the object. (addition of Gaussians)

In [102]:
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return new_mean, new_var

If our prior is 10 and 4, our motion is 12 and 4

In [103]:
predict(10., 4., 12., 4.)

(22.0, 8.0)

This is also the correct output. As we move our Gaussian, we are losing information and that depends on the variance of the motion Gaussian. Thus our variance increases.

### Main function

Like Localiztion, Kalman filter iterates through two things, measurement updates and motion updates.

In [104]:
measurements = [5., 6. ,7. , 9., 10.]
motion = [1.,1.,2.,1.,1.]
measurement_sig = 4.
motion_sig = 2.

In [105]:
def Kalman(measurements, motion, measurement_sig, motion_sig, mean = 0., sig = 10000.):
    for i in range(len(measurements)):
        print('On measurement {}'.format(measurements[i]))
        mean, sig = update(mean, sig, measurements[i], measurement_sig)
        print ('Update values: {}, {}\n'.format(mean, sig))
        print('On motion {}'.format(motion[i]))
        mean, sig = predict(mean, sig, motion[i], motion_sig)
        print ('Predict values: {}, {}\n'.format(mean, sig))

In [106]:
Kalman(measurements, motion, measurement_sig, motion_sig)

On measurement 5.0
Update values: 4.998000799680128, 3.9984006397441023

On motion 1.0
Predict values: 5.998000799680128, 5.998400639744102

On measurement 6.0
Update values: 5.999200191953932, 2.399744061425258

On motion 1.0
Predict values: 6.999200191953932, 4.399744061425258

On measurement 7.0
Update values: 6.999619127420922, 2.0951800575117594

On motion 2.0
Predict values: 8.999619127420921, 4.09518005751176

On measurement 9.0
Update values: 8.999811802788143, 2.0235152416216957

On motion 1.0
Predict values: 9.999811802788143, 4.023515241621696

On measurement 10.0
Update values: 9.999906177177365, 2.0058615808441944

On motion 1.0
Predict values: 10.999906177177365, 4.005861580844194



In Summary, the next predicted value for this array is 11.

### Explanation

To explain what is happenning, the mean of the probability the car is keeping a track of is being shifted as measurement values are coming in from the car's radar and sensors. As more measurements are being recieved from its sensors, the car is more certain of its location, which leads the variance of the subsequent gaussians to be less and the mean to be shifted to the correct location.

So this successfully predicted the next value this object will be at 11 with a variance which started with 10000, but it is has decreased drastically down to 4

## Kalman Filter to determine velocity

Kalman filters are also able to figure out the velocity of another object. From there it can make predictions for an object based on its velocity.

The variables for a Kalman filter represents the state of the physical world object such as where another car is and how fast it is moving.

There are two types of variables:
    - observable variables: such as location of the car
    - hidden variables: such as the velocity
    
Subsequent observations of the observable variables give us information about these hidden variables, so we can also estimate the hidden variables using Kalman Filters as well.


## 2d Kalman Filter

## Matrix class

Helper class to create matrices and provide matrix operations

In [81]:
from math import *
class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise (ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)

        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise (ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                    res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                    raise (ValueError, "Zero diagonal")
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)
    
    def __getitem__(self,index):
        return self.value[index]

@Credit to Sebastian Thrun for matrix helper function


## Variables

In [60]:
#initial state of location and velocity
x = matrix([[0.], [0.]])

#initial uncertainty of location and velocity
P = matrix([[1000., 0.], [0.,1000.]]) 

#external motion (we will leave as [[0.], [0.]])
u = matrix([[0.], [0.]])

#next state function (new location = location + velocity and velocity = velocity)
F = matrix([[1.,1], [0, 1.]])

#measurement function (to grab the measurement from matrix x)
H = matrix([[1., 0]])

#measurement uncertainty
R = matrix([[1.]])

#identity matrix
I = matrix([[1.,0.], [0., 1.]])

## Kalman filter function

We will perform a measurement update followed by a prediction for every measurement we recieve.

In a more practical example, the measurements will be measurements recieved from radar or laser sensors detecting an external object

In [96]:
def kalman_filter(x, P):
    for i in range(len(measurements)):
        # measurement update
        y = matrix([[measurements[i]]]) - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I - K * H) * P
        
        #prediction
        x = (F * x) + u
        P = (F * P) * F.transpose()
        print("Observed measurement: {}\n".format(measurements[i]))
        print("The predicted next location is: {:2f}".format(float(x[0][0])))
        print("The predicted velocity is : {:2f}".format(float(x[1][0])))
        print("The variance of the predicted next location is: {:2f}".format(P[0][0]))
        print("The variance of the predicted velocity is: {:2f}\n".format(P[1][1]))
    return x, P

Source: https://en.wikipedia.org/wiki/Kalman_filter

## Testing 

In [107]:
#measurement of locations observed from an object
measurements = [1,2,3,4,5]

print(kalman_filter(x, P))

Observed measurement: 1

The predicted next location is: 0.999001
The predicted velocity is : 0.000000
The variance of the predicted next location is: 1000.999001
The variance of the predicted velocity is: 1000.000000

Observed measurement: 2

The predicted next location is: 2.998003
The predicted velocity is : 0.999002
The variance of the predicted next location is: 4.990025
The variance of the predicted velocity is: 1.995013

Observed measurement: 3

The predicted next location is: 3.999666
The predicted velocity is : 1.000000
The variance of the predicted next location is: 2.331890
The variance of the predicted velocity is: 0.499501

Observed measurement: 4

The predicted next location is: 5.000000
The predicted velocity is : 1.000100
The variance of the predicted next location is: 1.499500
The variance of the predicted velocity is: 0.199870

Observed measurement: 5

The predicted next location is: 6.000100
The predicted velocity is : 1.000100
The variance of the predicted next loca

## Summary

In summary, given the only the first observed measurement of 1, the Kalman filter is unable to predict what the next location or velocity will be. This is because subsequent observations are needed to be able to estimate what the next point will be. Note that the variance is also still very high, as it is roughly our initial variance of 1000.

With a second observed measurement of 2. The Kalman filter is able to predict that the next location is ~3 and the predicted velocity of the object to be 1. Which is correct. Note that the variance of the predicted next location and the predicted velocity drops drastically with a second observed measurement.

With a third observed measurement of 3. The Kalman filter is able to predict the next location is ~4 and the predicted velocity is 1. Given the trajectory of the object, this is most likely the case. The variance of the predicted location and velocity is also alot lower then it was before.

The trend continues until the fifth observation.

Something to note is that the variance will keep dropping as long as new measurements are coming in. This is because the multiplication of two Gaussians always cause the variance to be smaller. So this behavior we are witnessing for the observed measurements is correct.

@Picture credits to Sebastian Thrund